<a href="https://colab.research.google.com/github/VasylDvorakDS/Picture_recognizing_with_CNN_-Mirrored-Strategy/blob/master/Picture_recognizing_with_CNN__Mirrored_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)



# additional imports

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model




Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.15.0


In [2]:


# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)



170498071/170498071 [==============================] - 4s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [3]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [4]:


# Build the model using the functional API
def create_model():
  i = Input(shape=x_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model



In [5]:


strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.experimental.CentralStorageStrategy()



In [6]:


print(f'Number of devices: {strategy.num_replicas_in_sync}')



Number of devices: 1


In [7]:


with strategy.scope():
  model = create_model()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])



In [8]:


# Fit
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)



Epoch 1/5
1563/1563 [==============================] - 556s 352ms/step - loss: 1.3000 - accuracy: 0.5544 - val_loss: 1.2896 - val_accuracy: 0.5591
Epoch 2/5
1563/1563 [==============================] - 538s 344ms/step - loss: 0.8297 - accuracy: 0.7118 - val_loss: 0.8546 - val_accuracy: 0.7103
Epoch 3/5
1563/1563 [==============================] - 547s 350ms/step - loss: 0.6795 - accuracy: 0.7669 - val_loss: 0.6707 - val_accuracy: 0.7717
Epoch 4/5
1563/1563 [==============================] - 562s 360ms/step - loss: 0.5718 - accuracy: 0.8027 - val_loss: 0.7213 - val_accuracy: 0.7644
Epoch 5/5
1563/1563 [==============================] - 557s 357ms/step - loss: 0.4849 - accuracy: 0.8348 - val_loss: 0.6432 - val_accuracy: 0.7932


In [9]:


50000/391



127.8772378516624

In [10]:


10000/79



126.58227848101266

In [11]:


# Compare this to non-distributed training
model2 = create_model()
model2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)



Epoch 1/5
1563/1563 [==============================] - 566s 358ms/step - loss: 1.3196 - accuracy: 0.5434 - val_loss: 1.0495 - val_accuracy: 0.6335
Epoch 2/5
1563/1563 [==============================] - 564s 361ms/step - loss: 0.8533 - accuracy: 0.7055 - val_loss: 0.9288 - val_accuracy: 0.6857
Epoch 3/5
1563/1563 [==============================] - 555s 355ms/step - loss: 0.6990 - accuracy: 0.7591 - val_loss: 0.7739 - val_accuracy: 0.7459
Epoch 4/5
1563/1563 [==============================] - 554s 355ms/step - loss: 0.5815 - accuracy: 0.7984 - val_loss: 0.8144 - val_accuracy: 0.7390
Epoch 5/5
1563/1563 [==============================] - 558s 357ms/step - loss: 0.4969 - accuracy: 0.8298 - val_loss: 0.7217 - val_accuracy: 0.7698
